In [ ]:
from PIL import Image
import numpy as np
from lang_sam import LangSAM
import lang_sam.utils as lsu
import matplotlib.pyplot as plt
import cv2
import io
import os

In [ ]:
model = LangSAM()

In [ ]:
image_pil = Image.open("/home/regal/devel/ws_cacti/src/hri_cacti_xr/gesture_recognition/gesture_recogition_research/action_clip_data/hl_frames/test/20240131_161505_d_rr_go_right/img_00121.jpg").convert("RGB")
text_prompt = "human-hand"
masks, boxes, phrases, logits = model.predict(image_pil, text_prompt)

In [ ]:
image_rgb = np.array(image_pil)
image = lsu.draw_image(image_rgb, masks, boxes, phrases)
plt.imshow(image)
plt.axis('off')  # Do not show axes to resemble an image
plt.show()

In [46]:
PrintDetails = True
# Path = '/home/regal/devel/ws_cacti/src/hri_cacti_xr/data/go_right/30x_4mono_10hz/20240131_162233_d_rr_go_right.avi'
Path = '/home/regal/devel/ws_cacti/src/hri_cacti_xr/data/go_right/30x_4mono_10hz/20240131_161505_d_rr_go_right.avi'
OutputPath = '/home/regal/devel/ws_cacti/src/hri_cacti_xr/gesture_recognition/gesture_recogition_research/lang-segment-anything/video.mp4'

# open video with opencv
cap = cv2.VideoCapture(Path)

# Get num frames, width, height, and frame rate
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width, frame_height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))

# create writer
out = cv2.VideoWriter(OutputPath, cv2.VideoWriter_fourcc(*"mp4v"), frame_rate, (frame_width, frame_height))

if(PrintDetails): 
    print(f"num frames: '{num_frames}'")
    print(f"frame rate: '{frame_rate}'")

#Loop through each frame in the video
while(cap.isOpened()):
    
    for num_frame in range(0, num_frames):
        
        # read the current frame as a numpy array
        ret, raw_frame = cap.read()

        # check if correctly returned
        if not ret:
            break
            
        # convert to image to pass into model
        pil_image = Image.fromarray(raw_frame)
            
        # determine depth from model (from run_video.py on GitHub)
        masks, boxes, phrases, logits = model.predict(pil_image, "human arm from an egocentric view")
        image = lsu.draw_image(raw_frame, masks, boxes, phrases)
        image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        img_resize = cv2.resize(image_bgr, (frame_width, frame_height))
        out.write(img_resize)
            
    break

cap.release()
out.release()

num frames: '176'
frame rate: '10'


/home/regal/miniconda3/envs/lsa/lib/python3.8/site-packages/transformers/modeling_utils.py:874: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/home/regal/miniconda3/envs/lsa/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
